In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import gc

gc.collect()

180

In [3]:
import json
import torch
from torch.optim import Adam
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.tensorboard.writer import SummaryWriter

from sklearn.model_selection import train_test_split

from common import data
from common.training import training_loop

from algos import deep_fib

from common.models import resnet, deeplab, scinet

SCINET = "scinet"
DEEPLAB = "deeplab"


In [4]:
paths = data.get_dataset_paths("../data")
train, test = train_test_split(paths, test_size=0.1, random_state=42)


In [5]:
m_data_train = data.Marconi100Dataset(train, scaling=data.Scaling.MINMAX)
m_data_test = data.Marconi100Dataset(test, scaling=data.Scaling.MINMAX)


Loading: 100%|██████████| 25/25 [00:39<00:00,  1.59s/it]


In [6]:
model_name = DEEPLAB
horizon = 1024
stride = 512
n_masks = 25

batch_size = 32

log_dir = "./outputs/df_dl_l1_25"
loss_type = "l1"
lr = 1e-3
num_epochs = 10

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


In [7]:
dataset_train = data.UnfoldedDataset(m_data_train, horizon=horizon, stride=stride)
dataset_test = data.UnfoldedDataset(m_data_test, horizon=horizon, stride=stride)

masks = deep_fib.get_masks(horizon, n_masks).float()

print(len(dataset_train), len(dataset_test), masks.size())


5644 647 torch.Size([25, 1024, 460])


In [8]:
train_loader = DataLoader(
    dataset_train,
    batch_size,
    shuffle=True,
)
test_loader = DataLoader(
    dataset_test,
    batch_size,
    shuffle=False,
)
print(len(train_loader), len(test_loader))


177 21


In [9]:
if model_name == SCINET:
    num_encoder_levels = 3
    hidden = [512]
    input_dim = 460
    hidden_size = 2
    kernel_size = 3
    dropout = 0.5

    model = scinet.SCINet(
        output_len=horizon,
        input_len=horizon,
        num_encoder_levels=num_encoder_levels,
        hidden_decoder_sizes=hidden,
        input_dim=input_dim,
        hidden_size=hidden_size,
        kernel_size=kernel_size,
        dropout=dropout,
    ).float()
    with open(f"{log_dir}/config.json", "w") as f:
        json.dump(
            dict(
                output_len=horizon,
                input_len=horizon,
                num_encoder_levels=num_encoder_levels,
                hidden_decoder_sizes=hidden,
                input_dim=input_dim,
                hidden_size=hidden_size,
                kernel_size=kernel_size,
                dropout=dropout,
            ),
            f,
        )
elif model_name == DEEPLAB:
    model = deeplab.DeepLabNet(
        resnet.ResNetFeatures(
            resnet.Bottleneck,
            resnet.RESNET50_LAYERS,
            return_layers=[resnet.LAYER_1, resnet.LAYER_4],
            replace_stride_with_dilation=[False, True, True],
            num_features=data.NUM_FEATURES,
        ),
        backbone_channels=[256, 2048],
        out_feats=data.NUM_FEATURES,
    ).float()
else:
    raise ValueError("Wrong model name")

print(type(model).__name__)


DeepLabNet


In [10]:
optim = Adam(model.parameters(), lr=lr)
lr_sched = CosineAnnealingLR(optim, num_epochs)
engine = deep_fib.DeepFIBEngine(
    model=model,
    device=device,
    optimizer=optim,
    lr_scheduler=lr_sched,
    masks=masks,
    loss_type=loss_type,
)

In [13]:
with SummaryWriter(log_dir) as writer:
    training_loop(
        engine=engine,
        num_epochs=num_epochs,
        train_dataloader=train_loader,
        test_dataloader=test_loader,
        writer=writer,
        save_path=log_dir + "/models",
        print_metrics=["auc"]
    )


Epoch 0 - train_loss = 0.052 - test_loss = 0.054 - test_auc=0.049 - lr = 1.00e-03


Epoch 1 - train_loss = 0.046 - test_loss = 0.054 - test_auc=0.048 - lr = 9.76e-04


Epoch 2 - train_loss = 0.044 - test_loss = 0.053 - test_auc=0.042 - lr = 9.05e-04


Epoch 3 - train_loss = 0.044 - test_loss = 0.054 - test_auc=0.041 - lr = 7.94e-04


KeyboardInterrupt: 